<a href="https://colab.research.google.com/github/Manish533005/nlp_chatbot/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
import numpy as np
from keras.models import load_model
import json
import random
from fuzzywuzzy import process
import spacy
model =load_model("chatbot_3.h5")
intents = json.loads(open('chatbot_dataset.json').read())
intents_1=json.loads(open("food_document.json").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
nlp=spacy.load("en_core_web_sm")
print(len(words))
print(len(classes))

44
14


In [ ]:
!pip install fuzzywuzzy

In [ ]:
print(intents_1)

{'intents': [{'tag': 'Non-Veg Pizza', 'items': [], 'size': [], 'price': []}, {'tag': 'Veg Pizza', 'items': [], 'size': [], 'price': []}, {'tag': 'Beverages', 'items': [], 'size': [], 'price': []}, {'tag': 'Slides', 'items': [], 'size': [], 'price': []}]}


In [ ]:
print(words)

['-', '-PRON-', 'add', 'available', 'beverage', 'bye', 'cancel', 'cancle', 'change', 'chat', 'chicken', 'city', 'combo', 'confirm', 'dominator', 'edit', 'food', 'good', 'goodbye', 'hello', 'hey', 'hi', 'hungry', 'item', 'large', 'later', 'medium', 'menu', 'nice', 'non', 'order', 'pizza', 'remove', 'request', 'slide', 'small', 'till', 'time', 'today', 'town', 'variety', 'veg', 'view', 'want']


In [ ]:
all_stopwords = nlp.Defaults.stop_words
sentence_2=[]
sent=[]

In [ ]:
#tokenize each word
def fuzzy(sentence):
    str2match=sentence
    strOptions=words
    Ratios=process.extract(str2match,strOptions)
    highest=process.extractOne(str2match,strOptions)
    return highest[0]
def clean_up_sentence(sentence):
        sentence=str(sentence)
        sentence_1=fuzzy(sentence)
        sentence_words= nlp(sentence_1)
        l=[]
        for i in sentence_words:
             i=str(i)
             l.append(i)
        sentence_2.extend(l)
        # lemmatizing the words
        sent=[nlp(word.lower())[0].lemma_ for word in sentence_2]
        sent=[ w  for w in sent if w not in all_stopwords]
        return sent
def bow(sentence ,words,show_details=True):

    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
     
    return return_list    

In [ ]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    print(tag)

    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    tag = ints[0]['intent']
    if tag=="order_food":
       k=print_menu()
       res=res+" \n "+ k[0] +" \n "+ k[1] +" \n "+ k[2] +" \n "+k[3]
    elif tag=="Non-Veg Pizza":
       k=print_nonveg()
       res=res+" \n "+ k[0] +" \n "+ k[1] +" \n "+ k[2] +" \n "+k[3]+" \n "

       
    return res

In [ ]:
k=chatbot_response("non veg")
print(k)


order_food
Explore our menu 
 Non-Veg Pizza 
 Veg Pizza 
 Beverages 
 Slides


In [ ]:
def print_nonveg():
    food=[]
    for intent in intents_1['intents']:
        for i in intent['items']:
            food.append(i)        
    return food
print_nonveg()         

['chicken dominator', 'chicken extravaganza']

In [ ]:
#method to print the menu
def print_menu():
  l=[]
  for intent in intents_1['intents']:
      l.append(intent['tag'])
  return l

In [ ]:
#method to print the non veg menu
def non_veg():
    for intent in intents_1['intents']:
      k=intent['tag']
      print(k)


In [ ]:
#Creating GUI with tkinter
import tkinter
from tkinter import *
import matplotlib


def send():
    msg = EntryBox.get("1.0",'end-1c').strip() 
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

base = Tk()
base.title("Domios Pizza chatbot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)